# Tutorial 2
In this tutorial we're going to take the model we developed in the previous tutorial, run it on the entire MNIST testing set and calculate the overall classification accuracy.

## Install PyGeNN wheel from Google Drive
Download wheel file

In [1]:
!gdown 1LMVTqYWWQdidyKKX-bKT-0EFTnzdFnm5

Downloading...
From: https://drive.google.com/uc?id=1LMVTqYWWQdidyKKX-bKT-0EFTnzdFnm5
To: /content/pygenn-4.9.0-cp310-cp310-linux_x86_64.whl
100% 20.6M/20.6M [00:00<00:00, 64.5MB/s]


and then install PyGeNN from wheel file

In [2]:
!pip install pygenn-4.9.0-cp310-cp310-linux_x86_64.whl

Processing ./pygenn-4.9.0-cp310-cp310-linux_x86_64.whl


Set environment variable to allow GeNN to find CUDA

In [3]:
%env CUDA_PATH=/usr/local/cuda

env: CUDA_PATH=/usr/local/cuda


## Download pre-trained weights and MNIST test data

In [4]:
!gdown 1cmNL8W0QZZtn3dPHiOQnVjGAYTk6Rhpc
!gdown 131lCXLEH6aTXnBZ9Nh4eJLSy5DQ6LKSF

Downloading...
From: https://drive.google.com/uc?id=1cmNL8W0QZZtn3dPHiOQnVjGAYTk6Rhpc
To: /content/weights_0_1.npy
100% 402k/402k [00:00<00:00, 4.85MB/s]
Downloading...
From: https://drive.google.com/uc?id=131lCXLEH6aTXnBZ9Nh4eJLSy5DQ6LKSF
To: /content/weights_1_2.npy
100% 5.25k/5.25k [00:00<00:00, 17.6MB/s]


## Install MNIST package

In [5]:
!pip install mnist

## Build model
As well as the standard modules and required PyGeNN functions and classes we used in the first tutorial, also import `time.perf_counter` for measuring the performance of our classifier and `tqdm.tqdm` for drawing progress bars

In [6]:
import mnist
import numpy as np
import matplotlib.pyplot as plt
from pygenn.genn_model import (create_custom_neuron_class,
                               create_custom_current_source_class,
                               GeNNModel)
from time import perf_counter
from tqdm.auto import tqdm

As before, define some simulation parameters

In [7]:
TIMESTEP = 1.0
PRESENT_TIMESTEPS = 100
INPUT_CURRENT_SCALE = 1.0 / 100.0

Create very similar neuron and current source models. However, to avoid having to download every spike and count them on the CPU, here, we add an additional state variable `SpikeCount` to each neuron which gets incremented in the reset code to count spikes.

In [8]:
# Very simple integrate-and-fire neuron model
if_model = create_custom_neuron_class(
    "if_model",
    param_names=["Vthr"],
    var_name_types=[("V", "scalar"), ("SpikeCount", "unsigned int")],
    sim_code="$(V) += $(Isyn) * DT;",
    reset_code="""
    $(V) = 0.0;
    $(SpikeCount)++;
    """,
    threshold_condition_code="$(V) >= $(Vthr)")

cs_model = create_custom_current_source_class(
    "cs_model",
    var_name_types=[("magnitude", "scalar")],
    injection_code="$(injectCurrent, $(magnitude));")

Build model, load weights and create neuron, synapse and current source populations as before

In [9]:
model = GeNNModel("float", "tutorial_2")
model.dT = TIMESTEP

# Load weights
weights_0_1 = np.load("weights_0_1.npy")
weights_1_2 = np.load("weights_1_2.npy")

if_params = {"Vthr": 5.0}
if_init = {"V": 0.0, "SpikeCount":0}
neurons = [model.add_neuron_population("neuron0", weights_0_1.shape[0],
                                       if_model, if_params, if_init),
           model.add_neuron_population("neuron1", weights_0_1.shape[1],
                                       if_model, if_params, if_init),
           model.add_neuron_population("neuron2", weights_1_2.shape[1],
                                       if_model, if_params, if_init)]
model.add_synapse_population(
        "synapse_0_1", "DENSE_INDIVIDUALG", 0,
        neurons[0], neurons[1],
        "StaticPulse", {}, {"g": weights_0_1.flatten()}, {}, {},
        "DeltaCurr", {}, {})
model.add_synapse_population(
        "synapse_1_2", "DENSE_INDIVIDUALG", 0,
        neurons[1], neurons[2],
        "StaticPulse", {}, {"g": weights_1_2.flatten()}, {}, {},
        "DeltaCurr", {}, {});

current_input = model.add_current_source("current_input", cs_model,
                                         neurons[0], {}, {"magnitude": 0.0})

Run code generator to generate simulation code for model and load it into PyGeNN as before but, here, we don't want to record any spikes so no need to specify a recording buffer size.

In [10]:
model.build()
model.load()

Just like in the previous tutorial, load testing images and labels and verify their dimensions

In [11]:
testing_images = mnist.test_images()
testing_labels = mnist.test_labels()

testing_images = np.reshape(testing_images, (testing_images.shape[0], -1))
assert testing_images.shape[1] == weights_0_1.shape[0]
assert np.max(testing_labels) == (weights_1_2.shape[1] - 1)

## Simulate model
In this tutorial we're going to not only inject current but also access the new spike count variable in the output population and reset the voltages throughout the model. Therefore we need to create some additional memory views

In [12]:
current_input_magnitude = current_input.vars["magnitude"].view
output_spike_count = neurons[-1].vars["SpikeCount"].view
neuron_voltages = [n.vars["V"].view for n in neurons]

Now, we define our inference loop. We loop through all of the testing images and for each one:

1.   Copy the (scaled) image data into the current input memory view and copy it to the GPU
2.   Loop through all the neuron populations, zero their membrance voltages and copy these to the GPU
3. Zero the output spike count and copy that to the GPU
4. Simulate the model for `PRESENT_TIMESTEPS`
5. Download the spike counts from the output layer
6. If highest spike count corresponds to correct label, increment `num_correct`



In [13]:
# Simulate
num_correct = 0
start_time = perf_counter()
for i in tqdm(range(testing_images.shape[0])):
    current_input_magnitude[:] = testing_images[i] * INPUT_CURRENT_SCALE
    current_input.push_var_to_device("magnitude")

    # Loop through all layers and their corresponding voltage views
    for l, v in zip(neurons, neuron_voltages):
        # Manually 'reset' voltage
        v[:] = 0.0

        # Upload
        l.push_var_to_device("V")

    # Zero spike count
    output_spike_count[:] = 0
    neurons[-1].push_var_to_device("SpikeCount")

    for t in range(PRESENT_TIMESTEPS):
        model.step_time()

    # Download spike count from last layer
    neurons[-1].pull_var_from_device("SpikeCount")

    # Find which neuron spiked the most to get prediction
    predicted_label = np.argmax(output_spike_count)
    true_label = testing_labels[i]

    if predicted_label == true_label:
        num_correct += 1

end_time = perf_counter()
print(f"\nAccuracy {((num_correct / float(testing_images.shape[0])) * 100.0)}%%")
print(f"Time {end_time - start_time} seconds")


  0%|          | 0/10000 [00:00<?, ?it/s]


Accuracy 97.44%%
Time 15.786295805999998 seconds
